In [ ]:
ческий алгоритм для подбора гиперпарамтеров классификатора

In [ ]:
!python --version

Python 3.7.15


In [ ]:
import xgboost as xgb
xgb.__version__

'0.90'

Исходно в Colab очень низкая версия библиотеки xgboost. Прийдется заменить.

ВНИМАНИЕ!!! Не забудьте ответить на запрос об отмене удаления


In [ ]:
%pip uninstall xgboost
%pip install xgboost==1.6.2
import xgboost as xgb
xgb.__version__

Found existing installation: xgboost 1.6.2
Uninstalling xgboost-1.6.2:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/xgboost-1.6.2.dist-info/*
    /usr/local/lib/python3.7/dist-packages/xgboost.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.7/dist-packages/xgboost/*
Proceed (y/n)? y
  Successfully uninstalled xgboost-1.6.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached xgboost-1.6.2-py3-none-manylinux2014_x86_64.whl (255.9 MB)


'1.6.2'

# RunAll. Запуск генетического алгоритма


In [ ]:
import numpy as np
import random
from numpy import loadtxt
from xgboost import XGBClassifier 
from xgboost.callback import EarlyStopping
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot
import os
import os.path
from tqdm.auto import tqdm
import glob
import warnings
from tensorflow.test import is_gpu_available
warnings.filterwarnings("ignore")

Colab = True
try:
    from google.colab import drive
except:
    Colab = False

if Colab:
    from google.colab import drive

    # Подключаем Google drive
    drive.mount('/content/drive')
    CrPath = "/content/drive/MyDrive/Uinnopolis/"

    import sys
    sys.path.append('/content/drive/MyDrive/Uinnopolis')
else:
    Acer = not os.path.exists("E:/Uinnopolis/")
    CrPath = "C:/Uinnopolis/" if Acer else "E:/Uinnopolis/"

import os, glob

from Libs import * #ReadCsv, WriteCsv, Graphic, Filter

from BaseGenetic import *

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
bgn v 3.0


In [ ]:
'''
    Метод на основании хромосомы запускает обучение модели и расчет результатов 
    валидационного и тестового ( публичного ) наборов
'''
def XGBoostHromosom(Hr, X, Y, ValidX, ValidY, XTest, Verbose=1, UseMetric="mlogloss", GPU = False):
    # Задаем параметры по хромосоме 
    RA = [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1, 5, 10, 50, 100]
    Methods = ['exact', 'approx', 'hist']
    CrMethod = Methods[int(Hr[10] * 3)]

    if GPU:  # and CrMethod == 'hist':
        CrMethod = 'gpu_hist'  # другие варианты не использую GPU

    print(1,1)
    model = XGBClassifier(learning_rate=0.03,
                          n_estimators= 100,#15000, #int(Hr[11] * 10000),
                          max_depth=int(Hr[4] * 7) + 3,
                          min_child_weight=int(Hr[5] * 5) + 1,
                          max_bin=int(Hr[0] * 100) + 5,
                          gamma=Hr[6] / 2,
                          subsample=Hr[7] * 0.4 + 0.6,
                          colsample_bytree=Hr[8] * 0.4 + 0.6,
                          reg_alpha=RA[int(Hr[9] * 14)],  # 0.005,
                          objective='binary:logistic',
                          nthread=8,
                          scale_pos_weight=1,
                          seed=int(Hr[3] * 65535),
                          tree_method=CrMethod,
                          random_state=int(Hr[3] * 65535),
                          verbose=1#Verbose
                          ,silent=True)

    # callback для раннего останова и запоминания лучшего состояния
    es = EarlyStopping(
        rounds=150,
        save_best=True,
        maximize=False,
        data_name="validation_0",
        metric_name=UseMetric
    )

    eval_set = [(ValidX, ValidY)]

    model.fit(X, Y, eval_metric=UseMetric, eval_set=eval_set,
              verbose=Verbose > 0)
    
    # make predictions for test data
    y_pred = model.predict(ValidX)

    # evaluate predictions
    accuracy = accuracy_score(ValidY, y_pred)
    print('Accuracy: %.2f%%' % (accuracy * 100.0))

    if XTest is not None and accuracy > 0.98:
        y_pred = model.predict(XTest)
        return accuracy, y_pred
    else:
        return accuracy, None


class TXGBGenetic(TBaseGenetic):  # TDistributedGenetic):
    def __init__(self, X, Y, ValidX, ValidY, XTest, YTest, 
                       Seed=None, StopFlag=2, TreeVerbose=0):
        TBaseGenetic.__init__(self, HromosomLen=12 + 2, FixedGroupsLeft=0,
                              StopFlag=StopFlag, TheBestListSize=100,
                              StartPopulationSize=50, PopulationSize=100)
        
        np.random.seed(Seed)
        self.X, self.Y = X, Y
        self.ValidX, self.ValidY = ValidX, ValidY
        self.XTest = XTest
        self.YTest = YTest
        #print('YTest', self.YTest)
        self.InverseMetric = False  # т.е. опимизация на возрастание
        self.Metric = 0
        self.StoredPath = f'{CrPath}copy/XGB3copy.dat'
        self.TreeVerbose = TreeVerbose
        self.TryLoadOnStart = True

    def TestHromosom(self, Hr, Id):
        if True:  # not Debug:
            
            CrValue, Test = XGBoostHromosom(Hr, self.X, self.Y, self.ValidX, self.ValidY, 
                                            self.XTest, GPU=self.GPU, Verbose=self.TreeVerbose)

            if Test is not None:
                WriteCsv(fr'{CrPath}XGB_5_{int(Id)}_{CrValue:.4f}.csv', self.YTest, Test)
                np.save(fr'{CrPath}XGB_5_{int(Id)}_{CrValue:.4f}.npy', Hr)
            else: print(1)
        else:
            CrValue = 1 - random.random() / 100
        return CrValue

    def GenerateHromosom(self, GetNewID=True):
        return np.random.random(size=self.HromosomLen)

    def Start(self):
        self.GPU = is_gpu_available()
        super().Start()


Готовим входные данные

In [ ]:
random.seed(25)

_x, Y = ReadCsv(CrPath, DelZeros=True, SortCtg=False, Train=True, RetPrc=True, PostProc=True)
y_valid = Y[:100]
y_train = Y[100:]

K = 20
y_train = np.tile(y_train, [K, 1])

XValid = np.load(f'{CrPath}Input/Valid_100.npy')
X = np.load(f'{CrPath}Input/Train_100.npy')

ResX0, YTest = ReadCsv(CrPath, DelZeros=True, SortCtg=False, Train=False, RetPrc=True, PostProc=True)
Ftr_Test = np.load(f'{CrPath}Input/Ftr_test.npy')
XTest = np.concatenate( (Ftr_Test, ResX0), -1 )

y_valid = np.tile(y_valid, [K, 1])

fInd = np.load(f'{CrPath}Input/FeatureIndex3.npy')
XTest = XTest[:, fInd]
#print(XTest.shape)
#print(y_train.shape)

In [ ]:
print(0,X.shape)
print(1,y_train.shape)
print(2,X.shape)
print(3,y_train[:, 1:2].shape) 
print(4,XValid.shape) 
print(5,y_valid[:, 1:2].shape) 
print(6,XTest.shape) 
print(7,YTest[:, 1:2].shape)
Gn = TXGBGenetic(X, y_train[:, 1:2], XValid, y_valid[:, 1:2], XTest, YTest = YTest[:, 1:2], StopFlag = 0, TreeVerbose = 0)
Gn.PDeath = 0.85
Gn.PMutation = 0.75 # probability of the mutation for any individuals. Or, if >=1, the number of individuals that will die in each generation
Gn.PMultiMutation = 0.3
Gn.PCrossingover = 0.5
Gn.Start()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Выходные данные были обрезаны до нескольких последних строк (5000).

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Accuracy: 97.20%
1
1 1
[16:11:13] WARNING: ../src/learner.cc:627: 
Parameters: { "scale_pos_weight", "silent", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


Accuracy: 96.60%
1
1 1
[16:11:19] WARNING: ../src/learner.cc:627: 
Parameters: { "scale_pos_weight", "silent", "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistak